In [4]:
import random

import numpy as np
import pandas as pd
import polars as pl
import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import resize

from src.conf import InferenceConfig, TrainConfig
from src.utils.common import (gaussian_label, nearest_valid_size,
                              negative_sampling, pad_if_needed, random_crop)


###################
# Label
###################
def get_seg_label(
    this_event_df: pd.DataFrame, num_frames: int, duration: int, start: int, end: int
) -> np.ndarray:
    # # (start, end)の範囲と(onset, wakeup)の範囲が重なるものを取得
    this_event_df = this_event_df.query("@start <= wakeup & onset <= @end")

    label = np.zeros((num_frames, 3))
    # onset, wakeup, sleepのラベルを作成
    for onset, wakeup in this_event_df[["onset", "wakeup"]].to_numpy():
        onset = int((onset - start) / duration * num_frames)
        wakeup = int((wakeup - start) / duration * num_frames)
        if onset >= 0 and onset < num_frames:
            label[onset, 1] = 1
        if wakeup < num_frames and wakeup >= 0:
            label[wakeup, 2] = 1

        onset = max(0, onset)
        wakeup = min(num_frames, wakeup)
        label[onset:wakeup, 0] = 1  # sleep

    return label


class SegTrainDataset(Dataset):
    def __init__(
        self,
        cfg: TrainConfig,
        features: dict[str, np.ndarray],
        event_df: pl.DataFrame,
    ):
        self.cfg = cfg
        # only positive samples
        self.event_df_pos: pd.DataFrame = (
            event_df.pivot(index=["series_id", "night"], columns="event", values="step")
            .drop_nulls()
            .to_pandas()
        )
        # all samples
        self.event_df: pd.DataFrame = (
            event_df.pivot(index=["series_id", "night"], columns="event", values="step")
            .to_pandas()
        )
        self.features = features
        self.num_features = len(cfg.features)
        self.upsampled_num_frames = nearest_valid_size(
            int(self.cfg.duration * self.cfg.upsample_rate), self.cfg.downsample_rate
        )
        self.max_steps = {'05e1944c3818': 139992, '13b4d6a01d27': 92484, 
                          '7476c0bd18d2': 83988, '5aad18e7ce64': 18216,
                          'aed3850f65f0': 21228, 'c5365a55ebb7': 17772, 
                          'f981a0805fd0': 75192}

    def __len__(self):
        return len(self.event_df)

    def __getitem__(self, idx):
        event = np.random.choice(["onset", "wakeup"], p=[0.5, 0.5])
        pos = self.event_df.at[idx, event]
        
        positive = False if random.random() < self.cfg.dataset.bg_sampling_rate else True
        if positive:
            idx = idx % len(self.event_df_pos)    
            series_id = self.event_df_pos.at[idx, "series_id"]
            this_event_df = self.event_df_pos.query("series_id == @series_id").reset_index(drop=True)
        else:
            series_id = self.event_df.at[idx, "series_id"]
            this_event_df = self.event_df.query("series_id == @series_id").reset_index(drop=True)
        
        # extract data matching series_id
        this_feature = self.features[series_id]  # (n_steps, num_features)
        n_steps = this_feature.shape[0]

        # sample background
        if positive:
            max_step = self.max_steps[series_id] if series_id in self.max_steps else 1e9
            pos = negative_sampling(this_event_df, n_steps, max_step, self.cfg.duration)

        # crop
        if n_steps > self.cfg.duration:
            start, end = random_crop(pos, self.cfg.duration, n_steps)
            feature = this_feature[start:end]
        else:
            start, end = 0, self.cfg.duration
            feature = pad_if_needed(this_feature, self.cfg.duration)

        # upsample
        feature = torch.FloatTensor(feature.T).unsqueeze(0)  # (1, num_features, duration)
        feature = resize(
            feature,
            size=[self.num_features, self.upsampled_num_frames],
            antialias=False,
        ).squeeze(0)

        # from hard label to gaussian label
        num_frames = self.upsampled_num_frames // self.cfg.downsample_rate
        label = get_seg_label(this_event_df, num_frames, self.cfg.duration, start, end)
        
        # label[:, 0] = add_gaussian_sleep(label[:, 0], offset=self.cfg.dataset.offset)
        label[:, [1, 2]] = gaussian_label(
            label[:, [1, 2]], offset=self.cfg.dataset.offset, sigma=self.cfg.dataset.sigma
        )
        
        return {
            "series_id": series_id,
            "feature": feature,  # (num_features, upsampled_num_frames)
            "label": torch.FloatTensor(label),  # (pred_length, num_classes)
        }


class SegValidDataset(Dataset):
    def __init__(
        self,
        cfg: TrainConfig,
        chunk_features: dict[str, np.ndarray],
        event_df: pl.DataFrame,
    ):
        self.cfg = cfg
        self.chunk_features = chunk_features
        self.keys = list(chunk_features.keys())
        self.event_df = (
            event_df.pivot(index=["series_id", "night"], columns="event", values="step")
            .drop_nulls()
            .to_pandas()
        )
        self.num_features = len(cfg.features)
        self.upsampled_num_frames = nearest_valid_size(
            int(self.cfg.duration * self.cfg.upsample_rate), self.cfg.downsample_rate
        )

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        feature = self.chunk_features[key]
        feature = torch.FloatTensor(feature.T).unsqueeze(0)  # (1, num_features, duration)
        feature = resize(
            feature,
            size=[self.num_features, self.upsampled_num_frames],
            antialias=False,
        ).squeeze(0)

        series_id, chunk_id = key.split("_")
        chunk_id = int(chunk_id)
        start = chunk_id * self.cfg.duration
        end = start + self.cfg.duration
        num_frames = self.upsampled_num_frames // self.cfg.downsample_rate
        
        label = get_seg_label(
            self.event_df.query("series_id == @series_id").reset_index(drop=True),
            num_frames,
            self.cfg.duration,
            start,
            end,
        )
        
        return {
            "key": key,
            "feature": feature,  # (batch_size, num_features, duration)
            "label": torch.FloatTensor(label),  # (batch_size, duration // 2, num_classes)
        }


class SegTestDataset(Dataset):
    def __init__(
        self,
        cfg: InferenceConfig,
        chunk_features: dict[str, np.ndarray],
    ):
        self.cfg = cfg
        self.chunk_features = chunk_features
        self.keys = list(chunk_features.keys())
        self.num_features = len(cfg.features)
        self.upsampled_num_frames = nearest_valid_size(
            int(self.cfg.duration * self.cfg.upsample_rate), self.cfg.downsample_rate
        )

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        feature = self.chunk_features[key]
        feature = torch.FloatTensor(feature.T).unsqueeze(0)  # (1, num_features, duration)
        feature = resize(
            feature,
            size=[self.num_features, self.upsampled_num_frames],
            antialias=False,
        ).squeeze(0)
        
        return {
            "key": key,
            "feature": feature,  # (num_features, duration)
        }


event                  onset    wakeup
series_id    night                    
038441c925bb 1        4992.0   10932.0
             2       20244.0   27492.0
             3       39996.0   44400.0
             4       57240.0   62856.0
             6       91296.0   97860.0
...                      ...       ...
fe90110788d2 30     505116.0  511284.0
             31     522852.0  529104.0
             32     538956.0  547152.0
             33     556560.0  560604.0
             34     574620.0  581604.0

[4790 rows x 2 columns]

# Ideas

- check  with fixed sampling 

- try with full data
- try with 6, 7, 8 folds

- try again gaussian loss
- fix DETR

What works and gives boost but overfits:
- LSTM encoder (0.772/~0.722)
- resnet50 (0.780/0.722)
- duration 17280 (0.772/0.722)

In [1]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from src.utils.metrics import event_detection_ap

from src.utils.post_process import post_process_for_seg
import jupyter_black
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import optuna


jupyter_black.load()
plt.style.use("ggplot")

INFERENCE = True
EXP_NAME = "transformer_dur_8640_folds"
RUN_NAME = "run0"
TYPE = "score"

if INFERENCE:
    RESULT_DIR = Path("../output/inference") / EXP_NAME  # / RUN_NAME
    hydra_result_dir = Path("../output/train") / "transformer_dur_8640_folds" / RUN_NAME
else:
    RESULT_DIR = Path("../output/train") / EXP_NAME / RUN_NAME
    hydra_result_dir = Path("../output/train") / EXP_NAME / RUN_NAME


def load_config(result_dir: Path):
    # clear previous initialization
    GlobalHydra.instance().clear()

    # initialize hydra
    config_path = result_dir / ".hydra"
    initialize(config_path=config_path.as_posix())
    # load the config
    cfg = compose(config_name="config")

    return cfg


cfg = load_config(hydra_result_dir)

if INFERENCE:
    preds = np.load(RESULT_DIR / f"preds.npy")
    keys = np.load(RESULT_DIR / f"keys.npy")
else:
    preds = np.load(RESULT_DIR / f"preds_{TYPE}.npy")
    labels = np.load(RESULT_DIR / f"labels_{TYPE}.npy")
    keys = np.load(RESULT_DIR / "keys.npy")

gt_df = pd.read_csv(Path(cfg.dir.data_dir) / "train_events.csv")
gt_df = gt_df[gt_df["series_id"].isin(cfg.split.valid_series_ids)].dropna().reset_index(drop=True)

/tmp/ipykernel_1800226/4092731947.py:40: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path=config_path.as_posix())


# Optimize postprocess parameters using Optuna

In [4]:
with open(Path(cfg.dir.processed_dir) / "train" / "series_lens.json") as f:
    series_lens = json.load(f)

pred_df = post_process_for_seg(keys, preds, series_lens, score_th=0.0001, distance=70, offset=8)
# pred_df = pred_df.to_pandas()

score = event_detection_ap(gt_df, pred_df)

score, len(pred_df)

(0.7945928237902742, 352926)

In [ ]:
# (0.7945928237902742, 352926)

In [18]:
# fold 5_0 - 0.765 / 0.733 - rank 1
# fold 5_3 - 0.772 / 0.733 - rank 3

# fold 10_2 - 0.0001 / 85 / 0.811 / 0.729 (0.709 for the best loss) - rank 6
# fold 10_4 - 0.0001 / 70 / 0.789 / 0.734 - rank 2
# fold 10_6 - 0.0001 / 70 / 0.779 / 0.733 - rank 4
# fold 10_7 - 0.0001 / 80 / 0.792 / 0.729 - rank 5
# fold 10_9 - 0.0001 / 80 / 0.812 / 0.722

In [ ]:
def objective(trial: optuna.Trial):
    # score_th = 0.005
    score_th = trial.suggest_float("score_th", 0, 0.5)
    distance = trial.suggest_int("distance", 30, 400)

    pred_df: pl.DataFrame = post_process_for_seg(keys, preds, score_th=score_th, distance=distance)
    score = event_detection_ap(gt_df, pred_df)

    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(study.best_params)

In [ ]:
def plot_random_sample(gt_df, keys, preds, labels, num_samples=1, num_chunks=10):
    # get series ids
    series_ids = np.array(list(map(lambda x: x.split("_")[0], keys)))
    unique_series_ids = np.unique(series_ids)

    # get random series
    random_series_ids = np.random.choice(unique_series_ids, num_samples)

    for i, random_series_id in enumerate(random_series_ids):
        # get random series
        series_idx = np.where(series_ids == random_series_id)[0]
        this_series_preds = preds[series_idx].reshape(-1, 3)
        this_series_labels = labels[series_idx].reshape(-1, 3)

        # split series
        this_series_preds = np.split(this_series_preds, num_chunks)
        this_series_labels = np.split(this_series_labels, num_chunks)
        this_series_len = [0] + [len(x) for x in this_series_labels]
        this_series_len = np.cumsum(this_series_len)

        gt_df = gt_df[gt_df["series_id"] == random_series_id]

        fig, axs = plt.subplots(num_chunks, 1, figsize=(20, 5 * num_chunks))

        if num_chunks == 1:
            axs = [axs]

        for j in range(num_chunks):
            this_series_preds_chunk = this_series_preds[j]
            this_series_labels_chunk = this_series_labels[j]

            # get onset and wakeup idx
            gt_tmp = gt_df[
                (gt_df["step"] >= this_series_len[j]) & (gt_df["step"] <= this_series_len[j + 1])
            ]
            onset_idx = gt_tmp.loc[gt_tmp["event"] == "onset", "step"].to_list()
            onset_idx = onset_idx - this_series_len[j]
            wakeup_idx = gt_tmp.loc[gt_tmp["event"] == "wakeup", "step"].to_list()
            wakeup_idx = wakeup_idx - this_series_len[j]

            axs[j].plot(this_series_preds_chunk[:, 0], label="pred_sleep")
            axs[j].plot(this_series_preds_chunk[:, 1], label="pred_onset")
            axs[j].plot(this_series_preds_chunk[:, 2], label="pred_wakeup")
            axs[j].vlines(onset_idx, 0, 1, label="onset", linestyles="dashed", color="C1")
            axs[j].vlines(wakeup_idx, 0, 1, label="wakeup", linestyles="dashed", color="C2")
            axs[j].set_ylim(0, 1)
            axs[j].set_title(f"series_id: {random_series_id} chunk_id: {j}")
            axs[j].legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)

    plt.tight_layout()


plot_random_sample(gt_df, keys, preds, labels, num_chunks=10)

# Inference

In [1]:
# Config
BEST_MODEL = "ensemble"
FOLD = 0

DURATION = 8640
DOWNSAMPLE_RATE = 2
PHASE = "train"
EXP_NAME = "transformer_best_folds"

In [2]:
!python -m run.inference\
    dir=local\
    model.params.encoder_name=resnet34\
    model.params.encoder_weights=null\
    num_workers=12\
    exp_name=$EXP_NAME\
    weight.run_name=single\
    batch_size=8\
    duration=$DURATION\
    downsample_rate=$DOWNSAMPLE_RATE\
    pp.score_th=0.0015\
    pp.distance=70\
    phase=$PHASE\
    best_model=$BEST_MODEL

Global seed set to 42
[5.0GB(+3.9GB):2.2sec] load test dataloader 
List of models: ['/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_0.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_2.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_4.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_3.pth']
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_0.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_2.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_4.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_3.pth
[5.7GB(+0.7GB):4.8sec] load model 
inference: 100%|████████████████████████████| 1867/1867 [02:38<00:00, 11.80it/s]
[7.4GB(+1.7GB):177.8sec] inference 
inference: 100%|████████████████████████████| 1867/1867 [02:38<00:00, 11.77it/s]
[7.4GB(+0.0GB):180.0sec] inference 
inf